<a href="https://colab.research.google.com/github/aminaalisheva/Unigrams-and-Sentiment-Classification-on-Kazakh-dataset/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Reviews - Sheet1 (1).csv to Reviews - Sheet1 (1).csv


In [ ]:
import pandas as pd

# Replace 'filename.csv' with your actual file name
df = pd.read_csv('Reviews.csv')
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.head()

,text,label
0,Көп рахмет супер әдемі демалдым,1
1,"Лаваштары суып қалған,кетчуптың орнына майонез...",0
2,"Консультант Айнур білікті маман, рақмет",1
3,"Глово арқылы заказ бергенбіз, фри піспеген. ша...",0
4,"Шаш қидыртуға барғанмын, мастері жаңа бастаған...",0


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer

# Lowercase conversion
df['text'] = df['text'].str.lower()

# Remove punctuation and special characters
def clean_text(text):
    return re.sub(r'[^а-яәғқңөұүһі\s]', '', text)

df['clean_text'] = df['text'].apply(clean_text)

# Tokenization using regex (your custom approach)
def custom_tokenize(text):
    # Example: split by spaces and merge hyphenated words
    tokens = re.findall(r'\b[\w-]+\b', text)
    # Merging duplicate hyphenated words (e.g., 'көк-шөп' -> 'көкшөп')
    tokens = [token.replace('-', '') for token in tokens]
    return tokens

df['tokens'] = df['clean_text'].apply(custom_tokenize)

# Remove stop words (optional - provide a Kazakh stopword list if you have one)
kazakh_stopwords = set(['мен', 'сен', 'ол', 'біз', 'сіз', 'олар', 'салон', 'мектеп', 'сабақ', 'тамақ', 'дәрігер', 'официант', 'және'])  # Example list
df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [t for t in tokens if t not in kazakh_stopwords])

# Join tokens back to strings for vectorization
df['final_text'] = df['filtered_tokens'].apply(lambda tokens: ' '.join(tokens))

# Feature extraction with Bag of Words
vectorizer = CountVectorizer()
X = df['final_text']
y = df['label']  # Labels from your dataset

print("Feature matrix shape:", X.shape)
df.head()

Feature matrix shape: (136,)


,text,label,clean_text,tokens,filtered_tokens,final_text
0,көп рахмет супер әдемі демалдым,1,көп рахмет супер әдемі демалдым,"[көп, рахмет, супер, әдемі, демалдым]","[көп, рахмет, супер, әдемі, демалдым]",көп рахмет супер әдемі демалдым
1,"лаваштары суып қалған,кетчуптың орнына майонез...",0,лаваштары суып қалғанкетчуптың орнына майонез ...,"[лаваштары, суып, қалғанкетчуптың, орнына, май...","[лаваштары, суып, қалғанкетчуптың, орнына, май...",лаваштары суып қалғанкетчуптың орнына майонез ...
2,"консультант айнур білікті маман, рақмет",1,консультант айнур білікті маман рақмет,"[консультант, айнур, білікті, маман, рақмет]","[консультант, айнур, білікті, маман, рақмет]",консультант айнур білікті маман рақмет
3,"глово арқылы заказ бергенбіз, фри піспеген. ша...",0,глово арқылы заказ бергенбіз фри піспеген шауы...,"[глово, арқылы, заказ, бергенбіз, фри, піспеге...","[глово, арқылы, заказ, бергенбіз, фри, піспеге...",глово арқылы заказ бергенбіз фри піспеген шауы...
4,"шаш қидыртуға барғанмын, мастері жаңа бастаған...",0,шаш қидыртуға барғанмын мастері жаңа бастаған ...,"[шаш, қидыртуға, барғанмын, мастері, жаңа, бас...","[шаш, қидыртуға, барғанмын, мастері, жаңа, бас...",шаш қидыртуға барғанмын мастері жаңа бастаған ...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text to Bag of Words (BoW)
vectorizer = CountVectorizer(max_features=5000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [ ]:
from collections import defaultdict

# Initialize a counter to track iterations
i = 0
word_counts_per_class = defaultdict(lambda: defaultdict(int))  # word -> count for each class
total_words_per_class = defaultdict(int)  # total word count per class
# Loop through the dataset and populate word counts per class
for doc, label in zip(X_train_bow, y_train):
    words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
    counts = doc.data
    for word_index, count in zip(words, counts):
        word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
        print(f"Index: {word_index}, Word: {word}, Count: {count}")
        word_counts_per_class[label][word] = count
        total_words_per_class[label] += count

    i += 1  # Increment the counter

    if i > 0:  # Stop after processing the first document
        break

print(word_counts_per_class[1]['жоқ'])
print(total_words_per_class[1])

Index: 78, Word: атмосферасы, Count: 1
Index: 945, Word: ұнады, Count: 3
Index: 915, Word: қызмет, Count: 2
Index: 478, Word: көрсетуі, Count: 1
Index: 357, Word: жоғары, Count: 1
Index: 567, Word: нәтижеде, Count: 1
Index: 444, Word: кухнясы, Count: 1
Index: 514, Word: маған, Count: 2
Index: 199, Word: бірақ, Count: 1
Index: 632, Word: салаттың, Count: 1
Index: 667, Word: соусы, Count: 1
Index: 11, Word: аз, Count: 1
Index: 180, Word: болып, Count: 2
Index: 931, Word: құрғақ, Count: 1
Index: 853, Word: қалды, Count: 1
Index: 354, Word: жолдасыма, Count: 1
Index: 799, Word: цомян, Count: 1
Index: 729, Word: тағамы, Count: 1
Index: 948, Word: ұнамады, Count: 1
Index: 834, Word: ішінде, Count: 1
Index: 435, Word: кинза, Count: 1
Index: 107, Word: бар, Count: 2
Index: 273, Word: екен, Count: 2
Index: 101, Word: балаларға, Count: 1
Index: 576, Word: ойын, Count: 1
Index: 34, Word: алаңы, Count: 1
Index: 103, Word: баламен, Count: 1
Index: 131, Word: барғанда, Count: 1
Index: 824, Word: ыңғ

In [ ]:
total_docs = len(y_train)
class_counts = y_train.value_counts()  # Count how many documents for each class docs_j
class_probs = {class_: count / total_docs for class_, count in class_counts.items()}  # P(c_j)
print(class_probs.keys(), class_probs[1])

dict_keys([1, 0]) 0.5185185185185185


In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

class NaiveBayes:
    def __init__(self):
        self.class_probs = {}  # Prior probabilities P(y)
        self.word_probs = defaultdict(lambda: defaultdict(float))  # Conditional probabilities P(x_i | y)
        self.vocab = set()  # Vocabulary (all unique words)

    def fit(self, X_train, y_train, vectorizer):
        """
        Train the Naive Bayes classifier using the training data.
        """
        # Store the vocabulary (list of words)
        self.vocab = set(vectorizer.get_feature_names_out())  # Store vocabulary from the vectorizer

        # Calculate the class probabilities (priors)
        total_docs = len(y_train)
        class_counts = y_train.value_counts()  # Count how many documents for each class docs_j
        self.class_probs = {class_: count / total_docs for class_, count in class_counts.items()}  # P(c_j)

        # Tokenize the words in X_train (already done by CountVectorizer)
        word_counts_per_class = defaultdict(lambda: defaultdict(int))  # word -> count for each class
        total_words_per_class = defaultdict(int)  # total word count per class

        # Loop through the dataset and populate word counts per class
        for doc, label in zip(X_train, y_train):
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            counts = doc.data
            for word_index, count in zip(words, counts):
                word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                word_counts_per_class[label][word] += count
                total_words_per_class[label] += count

        for label in word_counts_per_class:
            for word in word_counts_per_class[label]:
              self.word_probs[label][word] = (word_counts_per_class[label][word] + 1) / (total_words_per_class[label]+ len(self.vocab))

    def predict(self, X_test, vectorizer):
        """
        Predict labels for the given test data.
        """
        predictions = []
        for doc in X_test:
            log_probs = {}
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            # Compute the log probability for each class
            for label in self.class_probs.keys():
                log_prob = np.log(self.class_probs[label])
                for word_index in words:
                    word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                    log_prob += np.log(self.word_probs[label].get(word, 1 / (1000000*len(self.vocab))))

                log_probs[label] = log_prob

            # Choose the class with the highest log probability
            predictions.append(max(log_probs, key=log_probs.get))
        return predictions

# Initialize and train Naive Bayes model
nb_model = NaiveBayes()
nb_model.fit(X_train_bow, y_train, vectorizer)

# Make predictions on test data
y_pred = nb_model.predict(X_test_bow, vectorizer)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9285714285714286
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.86      1.00      0.92        12

    accuracy                           0.93        28
   macro avg       0.93      0.94      0.93        28
weighted avg       0.94      0.93      0.93        28



In [ ]:
sentiment_lexicon = {
    'жақсы': 1, 'керемет': 1, 'күшті': 1, 'шебері': 1, 'рахмет': 1,
    'жаман': -1, 'нашар': -1, '': -1, 'білікті': 1, 'дәмді': 1,
    'сыпайы': 1, 'жағымсыз': -1, 'ұзақ': -1, 'баяу': -1, 'кір': -1,
    'лас': -1, 'әдемі': 1, 'ұнады': 1, 'ұнамады': -1
}

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

class NaiveBayes:
    def __init__(self):
        self.class_probs = {}  # Prior probabilities P(y)
        self.word_probs = defaultdict(lambda: defaultdict(float))  # Conditional probabilities P(x_i | y)
        self.vocab = set()  # Vocabulary (all unique words)

    def get_sentiment_score(self, doc, lexicon):
        """
        Function to compute the sentiment score of a document based on a sentiment lexicon.
        Positive words will add +1, and negative words will subtract 1.
        """
        score = 0
        # Get the indices of non-zero values (words present in the BoW representation)
        words_indices = doc.nonzero()[1]  # This gives the indices of words that are present in the document
        feature_names = vectorizer.get_feature_names_out()  # Get the list of words in the vocabulary

        # Loop through the indices and check if the word is in the sentiment lexicon
        for word_index in words_indices:
          word = feature_names[word_index]  # Get the word corresponding to the index
          if word in lexicon:
            score += lexicon[word]  # Add the sentiment score for recognized words
        return score

    def fit(self, X_train, y_train, vectorizer):
        """
        Train the Naive Bayes classifier using the training data.
        """
        # Store the vocabulary (list of words)
        self.vocab = set(vectorizer.get_feature_names_out())  # Store vocabulary from the vectorizer

        # Calculate the class probabilities (priors)
        total_docs = len(y_train)
        class_counts = y_train.value_counts()  # Count how many documents for each class docs_j
        self.class_probs = {class_: count / total_docs for class_, count in class_counts.items()}  # P(c_j)

        # Tokenize the words in X_train (already done by CountVectorizer)
        word_counts_per_class = defaultdict(lambda: defaultdict(int))  # word -> count for each class
        total_words_per_class = defaultdict(int)  # total word count per class

        # Loop through the dataset and populate word counts per class
        for doc, label in zip(X_train, y_train):
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            counts = doc.data
            for word_index, count in zip(words, counts):
                word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                word_counts_per_class[label][word] += count
                total_words_per_class[label] += count

        for label in word_counts_per_class:
            for word in word_counts_per_class[label]:
              self.word_probs[label][word] = (word_counts_per_class[label][word] + 1 ) / (total_words_per_class[label]+ len(self.vocab))

    def predict(self, X_test, vectorizer):
        """
        Predict labels for the given test data.
        """
        predictions = []
        for doc in X_test:
            sentiment_score = self.get_sentiment_score(doc, sentiment_lexicon)
            log_probs = {}
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            # Compute the log probability for each class
            for label in self.class_probs.keys():
                log_prob = np.log(self.class_probs[label]) + np.log(sentiment_score + 1 + 1e-10)  # Add sentiment as an additional feature
                for word_index in words:
                    word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                    log_prob += np.log(self.word_probs[label].get(word, 1 / (1000000*len(self.vocab))))

                log_probs[label] = log_prob

            # Choose the class with the highest log probability
            predictions.append(max(log_probs, key=log_probs.get))
        return predictions

# Initialize and train Naive Bayes model
nb_model = NaiveBayes()
nb_model.fit(X_train_bow, y_train, vectorizer)

# Make predictions on test data
y_pred = nb_model.predict(X_test_bow, vectorizer)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9285714285714286
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.86      1.00      0.92        12

    accuracy                           0.93        28
   macro avg       0.93      0.94      0.93        28
weighted avg       0.94      0.93      0.93        28



Binary Naïve Bayes

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

class NaiveBayes:
    def __init__(self):
        self.class_probs = {}  # Prior probabilities P(y)
        self.word_probs = defaultdict(lambda: defaultdict(float))  # Conditional probabilities P(x_i | y)
        self.vocab = set()  # Vocabulary (all unique words)

    def get_sentiment_score(self, doc, lexicon):
        """
        Function to compute the sentiment score of a document based on a sentiment lexicon.
        Positive words will add +1, and negative words will subtract 1.
        """
        score = 0
        # Get the indices of non-zero values (words present in the BoW representation)
        words_indices = doc.nonzero()[1]  # This gives the indices of words that are present in the document
        feature_names = vectorizer.get_feature_names_out()  # Get the list of words in the vocabulary

        # Loop through the indices and check if the word is in the sentiment lexicon
        for word_index in words_indices:
          word = feature_names[word_index]  # Get the word corresponding to the index
          if word in lexicon:
            score += lexicon[word]  # Add the sentiment score for recognized words
        return score

    def fit(self, X_train, y_train, vectorizer):
        """
        Train the Naive Bayes classifier using the training data.
        """
        # Store the vocabulary (list of words)
        self.vocab = set(vectorizer.get_feature_names_out())  # Store vocabulary from the vectorizer

        # Calculate the class probabilities (priors)
        total_docs = len(y_train)
        class_counts = y_train.value_counts()  # Count how many documents for each class docs_j
        self.class_probs = {class_: count / total_docs for class_, count in class_counts.items()}  # P(c_j)

        # Tokenize the words in X_train (already done by CountVectorizer)
        word_counts_per_class = defaultdict(lambda: defaultdict(int))  # word -> count for each class
        total_words_per_class = defaultdict(int)  # total word count per class

        # Loop through the dataset and populate word counts per class
        for doc, label in zip(X_train, y_train):
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            counts = doc.data
            for word_index, count in zip(words, counts):
                word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                word_counts_per_class[label][word] += 1
                total_words_per_class[label] += 1

        for label in word_counts_per_class:
            for word in word_counts_per_class[label]:
              self.word_probs[label][word] = (word_counts_per_class[label][word] + 1 ) / (total_words_per_class[label]+ len(self.vocab))

    def predict(self, X_test, vectorizer):
        """
        Predict labels for the given test data.
        """
        predictions = []
        for doc in X_test:
            sentiment_score = self.get_sentiment_score(doc, sentiment_lexicon)
            log_probs = {}
            words = doc.nonzero()[1]  # Get the indices of non-zero values (the words present in the doc)
            # Compute the log probability for each class
            for label in self.class_probs.keys():
                log_prob = np.log(self.class_probs[label]) + np.log(sentiment_score + 1 + 1e-10)  # Add sentiment as an additional feature
                for word_index in words:
                    word = vectorizer.get_feature_names_out()[word_index]  # Get the word corresponding to the index
                    log_prob += np.log(self.word_probs[label].get(word, 1 / (1000000*len(self.vocab))))

                log_probs[label] = log_prob

            # Choose the class with the highest log probability
            predictions.append(max(log_probs, key=log_probs.get))
        return predictions

# Initialize and train Naive Bayes model
nb_model = NaiveBayes()
nb_model.fit(X_train_bow, y_train, vectorizer)

# Make predictions on test data
y_pred = nb_model.predict(X_test_bow, vectorizer)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8780487804878049
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.95      0.88        20
           1       0.94      0.81      0.87        21

    accuracy                           0.88        41
   macro avg       0.89      0.88      0.88        41
weighted avg       0.89      0.88      0.88        41



In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

class LogisticRegressionSoftmaxL1:
    def __init__(self, learning_rate=0.01, num_epochs=1000, reg_lambda=0.01):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.reg_lambda = reg_lambda
        self.weights = None

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def compute_loss(self, X, y_onehot):
        m = X.shape[0]
        logits = X @ self.weights
        probs = self.softmax(logits)

        # Cross-entropy loss
        cross_entropy = -np.mean(np.sum(y_onehot * np.log(probs + 1e-15), axis=1))

        # L1 regularization term
        l1_term = self.reg_lambda * np.sum(np.abs(self.weights))

        return cross_entropy + l1_term

    def fit(self, X, y):
        m, n = X.shape
        num_classes = np.max(y) + 1

        # One-hot encode labels (Updated: "sparse_output" instead of "sparse")
        encoder = OneHotEncoder(sparse_output=False)
        y_onehot = encoder.fit_transform(y.reshape(-1, 1))

        # Initialize weights
        self.weights = np.zeros((n, num_classes))

        for epoch in range(self.num_epochs):
            logits = X @ self.weights
            probs = self.softmax(logits)

            # Gradient with L1 regularization
            gradient = (X.T @ (probs - y_onehot)) / m + self.reg_lambda * np.sign(self.weights)

            # Update weights
            self.weights -= self.learning_rate * gradient

            if epoch % 100 == 0:
                loss = self.compute_loss(X, y_onehot)
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        logits = X @ self.weights
        probs = self.softmax(logits)
        return np.argmax(probs, axis=1)

# Example usage:
# vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['final_text']).toarray()
y = df['label'].values
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
lr_model = LogisticRegressionSoftmaxL1(learning_rate=0.05, num_epochs=2000, reg_lambda=0.01)
lr_model.fit(X_train, y_train)
#
y_pred = lr_model.predict(X_test)
#
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Epoch 0, Loss: 0.6911
Epoch 100, Loss: 0.5539
Epoch 200, Loss: 0.5125
Epoch 300, Loss: 0.4926
Epoch 400, Loss: 0.4812
Epoch 500, Loss: 0.4739
Epoch 600, Loss: 0.4680
Epoch 700, Loss: 0.4645
Epoch 800, Loss: 0.4614
Epoch 900, Loss: 0.4600
Epoch 1000, Loss: 0.4578
Epoch 1100, Loss: 0.4560
Epoch 1200, Loss: 0.4559
Epoch 1300, Loss: 0.4541
Epoch 1400, Loss: 0.4539
Epoch 1500, Loss: 0.4529
Epoch 1600, Loss: 0.4521
Epoch 1700, Loss: 0.4529
Epoch 1800, Loss: 0.4517
Epoch 1900, Loss: 0.4519
Accuracy: 0.8214285714285714
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.81      0.84        16
           1       0.77      0.83      0.80        12

    accuracy                           0.82        28
   macro avg       0.82      0.82      0.82        28
weighted avg       0.82      0.82      0.82        28



Task 4

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

# Replace 'filename.csv' with your actual file name
df_true = pd.read_csv('true_sent.csv')
df_true = df_true.sample(frac=1, random_state=42).reset_index(drop=True)

df_true.head()


,text,label
0,"Мен баобабтың бұта емес, шіркеудің биіктігімен...",1
1,"Мұрның қаншалықты ұзар са, сонша жаман қылық ж...",1
2,Себебі бұдан олардың тіршілігі түзеле қойған жоқ.,1
3,"Олар теңіз бетіне шығып, әртүрлі іс-қимылдар а...",1
4,"Мақұл деп үшеуі қатарласып, бір жерге келеді. ...",1


In [ ]:
import pandas as pd

# Replace 'filename.csv' with your actual file name
df_fake = pd.read_csv('fake_sent.csv')
df_fake = df_fake.sample(frac=1, random_state=42).reset_index(drop=True)

df_fake.head()

,text,label
0,жол телефон,0
1,кітап әжем қызық көлік саяхат аспан жабық ән к...,0
2,жабық көшеде адамдар кітап ... жол жол құстар ...,0
3,"жапырақ аспан ұл жол көшеде,",0
4,би қызық қар әжем ... кеш адамдар адамдар ағаш...,0


In [ ]:
import pandas as pd
df_true_sampled = df_true.sample(n=1500, random_state=42)
df_fake_sampled = df_fake.sample(n=1500, random_state=42)

# Convert text to lowercase and strip spaces to avoid small mismatches
df_fake["text"] = df_fake["text"].str.strip().str.lower()
df_true["text"] = df_true["text"].str.strip().str.lower()

df_true_sampled = df_true.sample(n=1500, random_state=42)
df_fake_sampled = df_fake.sample(n=1500, random_state=42)

# Merge using an outer join to keep all data, adding an indicator column
df_combined = pd.merge(df_fake_sampled, df_true_sampled, on=["text", "label"], how="outer", indicator=True)

# Keep only rows that do NOT appear in both (i.e., not "both")
df_combined = df_combined[df_combined["_merge"] != "both"].drop(columns=["_merge"])

# Shuffle the DataFrame
df_sampled = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_sampled.head())
df_sampled.to_csv('sentences.csv', index=False)
print(df_sampled["label"].value_counts())


                                                text  label
0   сонан соң тасты иемізге апарын береміз, – дейді.      1
1                                     күн саяхат тау      0
2  сонда сен барлық жұлдызды тамашалауды жақсы кө...      1
3  ашық достар қалам. қызық ашық қызық сабақ ойын...      0
4                        қыз қалам әжем қар адамдар.      0
label
1    1500
0    1500
Name: count, dtype: int64


In [ ]:
df_sampled = pd.read_csv('sentences.csv')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

class LogisticRegressionL1:
    def __init__(self, input_dim, learning_rate=0.001, lambda_l1=0.1, num_epochs=1000):
        self.learning_rate = learning_rate
        self.lambda_l1 = lambda_l1
        self.num_epochs = num_epochs
        self.weights = np.zeros(input_dim) * 0.01
        self.bias = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, y_true, y_pred):
        """
        Cross-entropy loss: -[y log(p) + (1 - y) log(1 - p)]
        """
        return -np.mean(y_true * np.log(y_pred + 1e-9) + (1 - y_true) * np.log(1 - y_pred + 1e-9))

    def l1_penalty(self):
        return self.lambda_l1 * np.sum(np.abs(self.weights))

    def fit(self, X, y):
        """
        Train using gradient descent with L1 regularization.
        """
        m = X.shape[0]

        for epoch in range(self.num_epochs):
            logits = np.dot(X, self.weights) + self.bias
            probs = self.sigmoid(logits)
            loss = self.compute_loss(y, probs) + self.l1_penalty()

            # Gradient calculation
            error = probs - y
            dw = (1 / m) * np.dot(X.T, error) + self.lambda_l1 * np.sign(self.weights)  # L1 gradient
            db = (1 / m) * np.sum(error)

            # Gradient update
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Predict class labels (0 or 1).
        """
        logits = np.dot(X, self.weights) + self.bias
        probs = self.sigmoid(logits)
        return (probs >= 0.5).astype(int)  # Convert to binary class

    # Convert text to BoW features
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df_sampled["text"]).toarray()
    y = np.array(df_sampled["label"])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = LogisticRegressionL1(input_dim=X_train.shape[1], learning_rate=0.01, lambda_l1=0.1, num_epochs=1000)
    model.fit(X_train, y_train)

       # Make predictions
    y_pred_prob = model.predict(X_test)  # This will give probabilities between 0 and 1
    y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions
    # Print Accuracy
    print("Accuracy:", accuracy_score(y_test, y_pred))

    # Print Classification Report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

NameError: name 'df_sampled' is not defined

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

class LogisticRegressionL2:
    def __init__(self, input_dim, learning_rate=0.01, lambda_l2=10.5, num_epochs=1000):
        self.learning_rate = learning_rate
        self.lambda_l2 = lambda_l2
        self.num_epochs = num_epochs
        self.weights = np.zeros(input_dim)  # Initialize weights to zero
        self.bias = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, y_true, y_pred):
        """
        Cross-entropy loss: -[y log(p) + (1 - y) log(1 - p)]
        """
        return -np.mean(y_true * np.log(y_pred + 1e-9) + (1 - y_true) * np.log(1 - y_pred + 1e-9))

    def l2_penalty(self):
        return self.lambda_l2 * np.sum(self.weights ** 2)

    def fit(self, X, y):
        """
        Train using gradient descent with L2 regularization.
        """
        m = X.shape[0]

        for epoch in range(self.num_epochs):
            logits = np.dot(X, self.weights) + self.bias
            probs = self.sigmoid(logits)
            loss = self.compute_loss(y, probs) + self.l2_penalty()

            # Gradient calculation
            error = probs - y
            dw = (1 / m) * np.dot(X.T, error) + 2 * self.lambda_l2 * self.weights  # L2 gradient
            db = (1 / m) * np.sum(error)

            # Gradient update
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        """
        Predict class labels (0 or 1).
        """
        logits = np.dot(X, self.weights) + self.bias
        probs = self.sigmoid(logits)
        return (probs >= 0.5).astype(int)  # Convert to binary class

# Example Usage
if __name__ == "__main__":
    # Example dataset: Texts & Labels (0 = No sentence boundary, 1 = Sentence boundary)
    data = {
        "text": [
            "Бұл оқиға өте қызықты.",
            "Мен ертең мектепке барамын",
            "Бүгін ауа райы керемет!",
            "Бұл оқиға өте қызықты. Мен ертең мектепке барамын",
            "Ауа райы керемет бірақ суық"
        ],
        "label": [1, 0, 1, 0, 0]  # Binary classification
    }

    # Convert text to BoW features
    df = pd.DataFrame(data)
    vectorizer = CountVectorizer(binary=True)
    X = vectorizer.fit_transform(df_sampled["text"]).toarray()
    y = np.array(df_sampled["label"])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = LogisticRegressionL2(input_dim=X_train.shape[1], learning_rate=0.01, lambda_l2=0.1, num_epochs=1000)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_prob = model.predict(X_test)  # This will give probabilities between 0 and 1
    y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions
    # Print Accuracy
    print("Accuracy:", accuracy_score(y_test, y_pred))

    # Print Classification Report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.6212
Epoch 200, Loss: 0.5863
Epoch 300, Loss: 0.5674
Epoch 400, Loss: 0.5563
Epoch 500, Loss: 0.5494
Epoch 600, Loss: 0.5449
Epoch 700, Loss: 0.5418
Epoch 800, Loss: 0.5397
Epoch 900, Loss: 0.5382
Accuracy: 0.9516666666666667
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       318
           1       0.91      1.00      0.95       282

    accuracy                           0.95       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.96      0.95      0.95       600



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1
from sklearn.metrics import accuracy_score, classification_report

# Build the model
model = Sequential([
    Dense(1, activation='sigmoid', input_dim=X_train.shape[1],
          kernel_regularizer=l1(0.01))  # For L1 regularization
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # You can adjust epochs and batch_size

# Make predictions
y_pred_prob = model.predict(X_test)  # This will give probabilities between 0 and 1
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Print Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7313 - loss: 1.0716
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9683 - loss: 0.6144
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9880 - loss: 0.5689
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9942 - loss: 0.5359
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9968 - loss: 0.5186
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9946 - loss: 0.5031
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9972 - loss: 0.4818
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9926 - loss: 0.4760
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9950 - loss: 0.4685
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9870 - loss: 0.4629
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Accuracy: 0.98
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97    

IndentationError: unexpected indent (<ipython-input-44-eaedae5ba29f>, line 2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, classification_report

# Build the model
model = Sequential([
    Dense(1, activation='sigmoid', input_dim=X_train.shape[1],
          kernel_regularizer=l2(0.01))  # For L1 regularization
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # You can adjust epochs and batch_size

# Make predictions
y_pred_prob = model.predict(X_test)  # This will give probabilities between 0 and 1
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Print Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7882 - loss: 0.6637
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9961 - loss: 0.5352
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9994 - loss: 0.4611
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9988 - loss: 0.4242
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9991 - loss: 0.3843
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9999 - loss: 0.3719
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9996 - loss: 0.3545
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9992 - loss: 0.3434
Epoch 9/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9999 - loss: 0.3338
Epoch 10/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9997 - loss: 0.3314
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00    